The model will classify the list of customers based on whether a customer will churn or not.In this data, false negative costs higher. If the customer is churned, but if we faslely predicted them as 'will not churn' then it costs higher.
We do not want to lose the customer by falsely identifying them as non-churning customer. So we will give more importance to recall compared to accuracy, precision and other factors.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

In [2]:
df=pd.read_csv('data\PreprocessedBankChurners.csv')

In [4]:
len(df.columns)

38

In [6]:
X = df.iloc[:, df.columns != 'Attrition_Flag']
y = df['Attrition_Flag']
X_train, X_test, y_train,y_test = train_test_split(X, y, test_size=0.25, random_state=21)

#### Logistic Regression

In [7]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression( )
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test) 
#(solvers=)

C:\Users\padma\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
print(confusion_matrix(y_test, y_pred))

[[2062   48]
 [ 221  201]]


In [9]:

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      2110
           1       0.81      0.48      0.60       422

    accuracy                           0.89      2532
   macro avg       0.86      0.73      0.77      2532
weighted avg       0.89      0.89      0.88      2532



Though accuracy is good, we need precision here. This is low for Attrited customer(48%)
So, we need to improve the model.
As there are many skewed data, since we get better results after standardization, we will standardize for Logistic Regression and tune the model

In [ ]:
X = df.iloc[:, df.columns != 'Attrition_Flag']
y = df['Attrition_Flag']
X_train, X_test, y_train,y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [18]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

###### Tuning the model

In [20]:
Cs = [0.001, 0.1, 1, 10, 100]
from sklearn.model_selection import GridSearchCV
param_grid = {'C': Cs}
logreg = LogisticRegression()
logreg_cv = GridSearchCV(logreg, param_grid, cv=5)

In [21]:
logreg_cv.fit(X_train_scaled, y_train)
print("Tuned parameter value = {}". format(logreg_cv.best_params_))
print('Best score = {}'. format(logreg_cv.best_score_))

Tuned parameter value = {'C': 10}
Best score = 0.9049374588545096


Best value for C is 10

In [24]:
logreg_best_param = LogisticRegression(C=10)
logreg_best_param.fit(X_train_scaled, y_train)
y_pred = logreg_best_param.predict(X_test_scaled) 

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.96      0.94      2113
           1       0.76      0.56      0.65       419

    accuracy                           0.90      2532
   macro avg       0.84      0.76      0.79      2532
weighted avg       0.89      0.90      0.89      2532



## Conclusion

The recall has increased from 48% to 56% but still is very low. So we will try other models.